# Setup Only

This notebook is not intended to run be by itself, but is a set up for the [deployment notebook](./01-deployment.ipynb). It is an abbreviated version of the [experiment annotaion notebook](../experiment/experiment_annotations.ipynb).  

In [ ]:
# imports and client instaniation
!pip install numpy
!pip install tensorflow
!pip install keras

import numpy
import pandas

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from cortex import Cortex

client = Cortex.client()
exp = client.experiment('flowers/experiment')

In [ ]:
# datasets
df = pandas.read_csv('../experiment/data/iris.data')
dataset = df.values
ds_classes = pandas.get_dummies(df).values # one-hot encode the classes

X = dataset[:,0:4].astype(float)
Y = ds_classes[:,4:8].astype(int)

(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(X, Y, test_size=0.333, train_size=0.667)

In [ ]:
# models
lf = 'categorical_crossentropy'

def adam_opt_model():
    model = Sequential()
    model.name = 'adam optimizer'
    model.add(Dense(16, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss=lf, optimizer='adam', metrics=['accuracy'])
    return model


def small_sgd_opt_model():
    model = Sequential()
    model.name = 'small sgd optimizer'
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss=lf, optimizer='sgd', metrics=['accuracy'])
    return model

In [ ]:
epoch_cts = [1, 10]
batch_sz = 3

for model in [adam_opt_model, small_sgd_opt_model]:
    for epoch_ct in epoch_cts:
        with exp.start_run() as run: 
            estimator = KerasClassifier(build_fn=model, epochs=epoch_ct, batch_size=batch_sz, verbose=0)
            kfold = KFold(n_splits=3, shuffle=True)
            results = cross_val_score(estimator, test_inputs, test_classes, cv=kfold)
            run.log_artifact('model', model())
            run.log_param('batch size', batch_sz)
            run.log_param('epochs', epoch_ct)
            run.log_param('model name', model().name)
            run.log_metric('mean acc', results.mean()*100)
            run.log_metric('margin of err', results.std()*100)